In [ ]:
import os
import tarfile
import requests

def download_and_extract(url, data_path='/Users/space/Desktop/cnn'):
    tgz_path = os.path.join(data_path, 'imagenette.tgz')
    if not os.path.exists(data_path):
        os.makedirs(data_path)
        print("Directory created:", data_path)
    if not os.path.isfile(tgz_path):
        print("Downloading...", end=" ")
        response = requests.get(url, stream=True)
        with open(tgz_path, 'wb') as f:
            for chunk in response.iter_content(chunk_size=128):
                f.write(chunk)
        print("Done.")
    print("Extracting...", end=" ")
    with tarfile.open(tgz_path, 'r:gz') as tar:
        tar.extractall(path=data_path)
    os.remove(tgz_path)
    print("Extraction complete.")


# URL for Imagenette (you can choose a different size dataset if needed)
url = 'https://s3.amazonaws.com/fast-ai-imageclas/imagenette2-320.tgz'
download_and_extract(url)


Directory created: /Users/space/Desktop/cnn
Downloading... Done.
Extracting... Extraction complete.


In [ ]:
pip install pytorch_lightning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 801.9/801.9 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 841.5/841.5 kB 56.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-man

In [ ]:
import os
import pytorch_lightning as pl
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

class ImagenetteDataModule(pl.LightningDataModule):
    def __init__(self, data_dir='/home/user/Desktop/CNN4', batch_size=64):
        super().__init__()
        self.data_dir = data_dir
        self.batch_size = batch_size

    def setup(self, stage=None):
        transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
        self.train_dataset = datasets.ImageFolder(os.path.join(self.data_dir, 'train'), transform=transform)
        self.val_dataset = datasets.ImageFolder(os.path.join(self.data_dir, 'val'), transform=transform)

    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True)

    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.batch_size)


BASIC CNN MODEL

In [ ]:

# Reinstall PyTorch
!pip install torch torchvision torchaudio


In [ ]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from pytorch_lightning import LightningModule, Trainer
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
from pytorch_lightning.callbacks import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger

class OBCNN(LightningModule):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)  # Reduced number of filters
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)  # Reduced number of filters
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(32 * 28 * 28, 64)  # Reduced size and complexity
        self.fc2 = nn.Linear(64, 10)  # Assuming 10 classes in Imagenette

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

    def training_step(self, batch, batch_idx):
        images, labels = batch
        outputs = self(images)
        loss = F.cross_entropy(outputs, labels)
        self.log('train_loss', loss, on_step=False, on_epoch=True)
        return loss

    def validation_step(self, batch, batch_idx):
        images, labels = batch
        outputs = self(images)
        val_loss = F.cross_entropy(outputs, labels)
        self.log('val_loss', val_loss, on_step=False, on_epoch=True)
        return val_loss

    def test_step(self, batch, batch_idx):
        images, labels = batch
        outputs = self(images)
        test_loss = F.cross_entropy(outputs, labels)
        self.log('test_loss', test_loss, on_step=False, on_epoch=True)
        preds = torch.argmax(outputs, dim=1)
        accuracy = torch.tensor(torch.sum(preds == labels).item() / len(preds))
        self.log('test_accuracy', accuracy, on_step=False, on_epoch=True)
        return {"test_loss": test_loss, "test_accuracy": accuracy}

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=0.001)
        return optimizer

    def setup(self, stage=None):
        transform = transforms.Compose([
            transforms.Resize((112, 112)),  # Reduced resolution
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
        dataset_path = '/Users/space/Desktop/cnn'
        full_dataset = datasets.ImageFolder(os.path.join(dataset_path), transform=transform)
        train_size = int(0.8 * len(full_dataset))
        test_val_size = len(full_dataset) - train_size
        val_size = test_val_size // 2
        test_size = test_val_size - val_size
        self.train_dataset, self.val_dataset, self.test_dataset = random_split(full_dataset, [train_size, val_size, test_size])

    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=32, shuffle=True)

    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=32, shuffle=False)

    def test_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=32, shuffle=False)

# Ensure the logging directory exists
log_dir = '/Users/space/Desktop/logs'
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

# Setup and training
logger = TensorBoardLogger(log_dir, name='OBCNN')
trainer = Trainer(
    max_epochs=10,
    logger=logger,
    callbacks=[EarlyStopping(monitor='val_loss', patience=3)]
)
model = OptimizedBasicCNN()
trainer.fit(model)
trainer.test(model)


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type      | Params
------------------------------------
0 | conv1 | Conv2d    | 448   
1 | conv2 | Conv2d    | 4.6 K 
2 | pool  | MaxPool2d | 0     
3 | fc1   | Linear    | 1.6 M 
4 | fc2   | Linear    | 650   
------------------------------------
1.6 M     Trainable params
0         Non-trainable params
1.6 M     Total params
6.446     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │            1.0            │
│         test_loss         │            0.0            │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.0, 'test_accuracy': 1.0}]

ALL CONVOLUTION NET

In [ ]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from pytorch_lightning import LightningModule, Trainer
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
from pytorch_lightning.callbacks import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger

class ACL(LightningModule):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 96, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(96, 96, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(96, 96, kernel_size=3, stride=2, padding=1)
        self.conv4 = nn.Conv2d(96, 192, kernel_size=3, padding=1)
        self.conv5 = nn.Conv2d(192, 192, kernel_size=3, padding=1)
        self.conv6 = nn.Conv2d(192, 192, kernel_size=3, stride=2, padding=1)
        self.conv7 = nn.Conv2d(192, 192, kernel_size=3, padding=1)
        self.conv8 = nn.Conv2d(192, 192, kernel_size=1)
        self.class_conv = nn.Conv2d(192, 10, kernel_size=1)  # Assuming 10 classes

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = F.relu(self.conv5(x))
        x = F.relu(self.conv6(x))
        x = F.relu(self.conv7(x))
        x = F.relu(self.conv8(x))
        x = F.relu(self.class_conv(x))
        x = F.avg_pool2d(x, x.size()[2:]).view(x.size(0), -1)
        return x

    def training_step(self, batch, batch_idx):
        images, labels = batch
        outputs = self(images)
        loss = F.cross_entropy(outputs, labels)
        self.log('train_loss', loss)
        return loss

    def validation_step(self, batch, batch_idx):
        images, labels = batch
        outputs = self(images)
        val_loss = F.cross_entropy(outputs, labels)
        self.log('val_loss', val_loss)
        return {'val_loss': val_loss}

    def test_step(self, batch, batch_idx):
        images, labels = batch
        outputs = self(images)
        loss = F.cross_entropy(outputs, labels)
        preds = torch.argmax(outputs, dim=1)
        accuracy = torch.sum(preds == labels).item() / len(labels)
        self.log('test_loss', loss)
        self.log('test_accuracy', accuracy)
        return {'test_loss': loss, 'test_accuracy': accuracy}

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.001)

    def setup(self, stage=None):
        transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
        dataset_path = '/Users/space/Desktop/cnn'
        dataset = datasets.ImageFolder(dataset_path, transform=transform)
        train_size = int(0.8 * len(dataset))
        test_val_size = len(dataset) - train_size
        test_size = test_val_size // 2
        val_size = test_val_size - test_size
        self.train_dataset, self.val_dataset, self.test_dataset = random_split(dataset, [train_size, val_size, test_size])

    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=64, shuffle=True)

    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=64, shuffle=False)

    def test_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=64, shuffle=False)

# Trainer Setup
logger = TensorBoardLogger('tb_logs', name='ACL')
trainer = Trainer(
    max_epochs=10,
    callbacks=[EarlyStopping(monitor='val_loss')],
    logger=logger
)
model = ACL()
trainer.fit(model)
trainer.test(model)


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name       | Type   | Params
--------------------------------------
0 | conv1      | Conv2d | 2.7 K 
1 | conv2      | Conv2d | 83.0 K
2 | conv3      | Conv2d | 83.0 K
3 | conv4      | Conv2d | 166 K 
4 | conv5      | Conv2d | 331 K 
5 | conv6      | Conv2d | 331 K 
6 | conv7      | Conv2d | 331 K 
7 | conv8      | Conv2d | 37.1 K
8 | class_conv | Conv2d | 1.9 K 
--------------------------------------
1.4 M     Trainable params
0         Non-trainable params
1.4 M     Total params
5.479     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │            1.0            │
│         test_loss         │            0.0            │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.0, 'test_accuracy': 1.0}]

In [ ]:
!pip install pytorch-lightning


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 801.9/801.9 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 841.5/841.5 kB 10.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-many

REGULARIZATION

In [ ]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from pytorch_lightning import LightningModule, Trainer
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
from pytorch_lightning.callbacks import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger

class RACL(LightningModule):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 96, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(96, 96, kernel_size=3, stride=2, padding=1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.conv3 = nn.Conv2d(96, 192, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(192, 192, kernel_size=3, stride=2, padding=1)
        self.dropout2 = nn.Dropout2d(0.25)
        self.conv5 = nn.Conv2d(192, 192, kernel_size=1)
        self.class_conv = nn.Conv2d(192, 10, kernel_size=1)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.dropout1(x)
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = self.dropout2(x)
        x = F.relu(self.conv5(x))
        x = F.relu(self.class_conv(x))
        x = F.avg_pool2d(x, x.size()[2:]).view(x.size(0), -1)
        return x

    def training_step(self, batch, batch_idx):
        images, labels = batch
        outputs = self(images)
        loss = F.cross_entropy(outputs, labels)
        self.log('train_loss', loss, on_step=False, on_epoch=True)
        return loss

    def validation_step(self, batch, batch_idx):
        images, labels = batch
        outputs = self(images)
        val_loss = F.cross_entropy(outputs, labels)
        self.log('val_loss', val_loss, on_step=False, on_epoch=True)
        return {'val_loss': val_loss}

    def test_step(self, batch, batch_idx):
        images, labels = batch
        outputs = self(images)
        loss = F.cross_entropy(outputs, labels)
        preds = torch.argmax(outputs, dim=1)
        accuracy = torch.sum(preds == labels).item() / len(labels)
        self.log('test_loss', loss, on_step=False, on_epoch=True)
        self.log('test_accuracy', accuracy, on_step=False, on_epoch=True)
        return {'test_loss': loss, 'test_accuracy': accuracy}

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.001)

    def setup(self, stage=None):
        transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.RandomHorizontalFlip(),  # Data Augmentation
            transforms.RandomRotation(10),  # Data Augmentation
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
        data_folder_path = '/Users/space/Desktop/cnn'
        dataset = datasets.ImageFolder(data_folder_path,transform=transform)
        train_size = int(0.8 * len(dataset))
        test_val_size = len(dataset) - train_size
        val_size = test_val_size // 2
        test_size = test_val_size - val_size
        self.train_data, self.val_data, self.test_data = random_split(dataset, [train_size, val_size, test_size])

    def train_dataloader(self):
        return DataLoader(self.train_data, batch_size=64, shuffle=True)

    def val_dataloader(self):
        return DataLoader(self.val_data, batch_size=64, shuffle=False)

    def test_dataloader(self):
        return DataLoader(self.test_data, batch_size=64, shuffle=False)

# Setup and training
tensorboard_logger = TensorBoardLogger('tb_logs', name='ACL')
training_manager = Trainer(
    max_epochs=10,
    callbacks=[EarlyStopping(monitor='val_loss')],
   logger=tensorboard_logger,
    accelerator='auto',  # Utilize GPU if available
    devices=1            # Use 1 GPU
)
model = RACL()
training_manager.fit(model)
training_manager.test(model)


INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.callbacks.model_summary:
  | Name       | Type      | Params
-----------------------------------------
0 | conv1      | Conv2d    | 2.7 K 
1 | conv2      | Conv2d    | 83.0 K
2 | dropout1   | Dropout2d | 0     
3 | conv3      | Conv2d    | 166 K 
4 | conv4      | Conv2d    | 331 K 
5 | dropout2   | Dropout2d | 0     
6 | conv5      | Conv2d    | 37.1 K
7 | class_conv | Conv2d    | 1.9 K 
-----------------------------------------
622 K     Trainable params
0         Non-trainable params
622 K     Total params
2.491     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

In [ ]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models, datasets, transforms
from torch.utils.data import DataLoader, random_split
from pytorch_lightning import LightningModule, Trainer
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger
import torchmetrics
import multiprocessing as mp

# Set multiprocessing to 'spawn' to avoid conflicts in multithreaded environments
mp.set_start_method('spawn', force=True)

class TransferLearningModel(LightningModule):
    def __init__(self, base_model, num_classes=10):
        super().__init__()
        # Replace the classifier in ResNet18
        self.feature_extractor = nn.Sequential(*list(base_model.children())[:-1])  # Extract features
        self.classifier = nn.Linear(base_model.fc.in_features, num_classes)  # New classifier for CIFAR-10

    def forward(self, x):
        x = self.feature_extractor(x)
        x = torch.flatten(x, 1)  # Flatten the features for the classifier
        x = self.classifier(x)
        return x

    def training_step(self, batch, batch_idx):
        images, labels = batch
        outputs = self(images)
        loss = F.cross_entropy(outputs, labels)
        self.log('train_loss', loss)
        return loss

    def validation_step(self, batch, batch_idx):
        images, labels = batch
        outputs = self(images)
        val_loss = F.cross_entropy(outputs, labels)
        self.log('val_loss', val_loss)
        return {'val_loss': val_loss}

    def test_step(self, batch, batch_idx):
        images, labels = batch
        outputs = self(images)
        loss = F.cross_entropy(outputs, labels)
        self.log('test_loss', loss)
        return {'test_loss': loss}

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.001)

    def setup(self, stage=None):
        # CIFAR-10
        transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
        ])
        if stage == 'fit' or stage is None:
            dataset = datasets.CIFAR10(root='.', train=True, download=True, transform=transform)
            train_size = int(0.8 * len(dataset))
            val_size = len(dataset) - train_size
            self.train_dataset, self.val_dataset = random_split(dataset, [train_size, val_size])
        if stage == 'test' or stage is None:
            self.test_dataset = datasets.CIFAR10(root='.', train=False, download=True, transform=transform)

    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=64, shuffle=True)

    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=64, shuffle=False)

    def test_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=64, shuffle=False)

# Load a pre-trained model
pre_trained_model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)

# Initialize the model for CIFAR-10
model = TransferLearningModel(pre_trained_model)

# Trainer configuration
trainer = Trainer(
    max_epochs=10,
    callbacks=[EarlyStopping(monitor='val_loss')],
    accelerator='auto',
    devices=1 if torch.cuda.is_available() else None  # Use 1 GPU if available
)

# Start training and testing
trainer.fit(model)
trainer.test(model)
